<a href="https://colab.research.google.com/github/macgyver121/Project-Chart-to-Text-generation/blob/main/pytessaract_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (4,333 kB/s)
debconf: unable to initi

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
import cv2
import pytesseract
import shutil
import os
import random
from PIL import Image

import re
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle

with open('List_key_ielts.pkl', 'rb') as f:
    List_key = pickle.load(f)

In [ ]:
for i in range(len(List_key)) :
    print(i+2 , List_key[i])

2 Holiday leisure activities Number of hours |At the beach Reading books Chinese tourists 3 8 American tourists 4 5. Turkish tourists 5 4
3 See aan Average income per person ($) | 2000 | 17,000 | 22,000 |
4 The table shows data about underground railway systems in six major cities with date opened, kilometres of route and passenger numbers per year in millions. Summarise the information by selecting and reporting the main features, making comparisons where relevant City | Date opened Kilometres of route coerce come London 1863 394 775 Paris 1900 199 1191 Tokyo 1927 155 1928 Washington DC 1976 126 144 Kyoto 1981 a} 45
5 The table gives information about department and online stores in Australia in 2011. Summarise the information by selecting and reporting the main features, and make comparisons where relevant. Department Stores Online Stores Number of Businesses 67 368 Profit (AUD) 807 863 Sales Revenue (AUD) 12,700 13,400
6 Spend Average Spend Average Nights (millions) (poundbillions) 

In [ ]:
Ref_key = [
    ["Holiday leisure activities Number of hours At the beach Reading books Chinese tourists 3 8 American tourists 4 5 Turkish tourists 5 4 Brazilian tourists 6 4"] , 
    ['Average income per person ($) 2000 17,000 22,000 '],
    ["The table shows data about underground railway systems in six major cities with date opened, kilometres of route and passenger numbers per year in millions. Summarise the information by selecting and reporting the main features, making comparisons where relevant City Date opened Kilometres of route Passengers per year London 1863 394 775 Paris 1900 199 1191 Tokyo 1927 155 1928 Washington DC 1976 126 144 Kyoto 1981 11 45"],
    ["The table gives information about department and online stores in Australia in 2011. Summarise the information by selecting and reporting the main features, and make comparisons where relevant. Department Stores Online Stores Number of Businesses 67 368 Profit (AUD) 807 863 Sales Revenue (AUD) 12,700 13,400"],
    ["Spend Average Spend Average Nights (millions) (poundbillions) _per Visits per Visits. 2003 24715 11.855 475 a2 2004 20.755 13.047 486 a2 2005 2971 14.248 an a3 2006 3273 16.002 486 a4 2007 3278 15.960 487 7"],
    ["Milk Full Fat Low Fat Butter Full Fat Low Fat 1200ml 59% 41% 12g AGES 1650ml 20% 38% 62% 45-65 1670ml 45% 55% 50% 50% 1900ml 30% 70% 30g, 49%"],
    ["Department Stores Online Stores Number of Businesses 67 368 Profit (AUD) 807 863 Sales Revenue (AUD) 12,700 13,400 Growth .4% .6%"],
    ["Independent films releases in the UK and Republic of Irland by genre 2012 Genre Number of % of allreleases % of income from releases ticket sales Comedy 26 17.6 45.4 Horror 14 9.5 20.2 Biopic 1 0.7 9.1 Drama 35 23.6 83 Crime 7 47 47 Action 4 27 41 Documentary 35 23.6 2.9 Thriller B 88 13 Romance 5 34 08 Other 8 5.4 3.2"], 
    ["$HK billions Equipment Telecommunications Clothing Manufacturing Metals Overall 2009 10.3 7.9 5.5 23 32 2010 11.6 12.7 51 38.4 Changes 13% 61% -17% -27% 122%"],
    ["France USA Spain UK Italy Turkey China Germany Hong Kong in 2009 76 55 55 32 in 2010 76 67 56 57 45 24 % Change 3.6 23 33 55"],
    ["The table gives information about five types of vehicles registered in Australia in 2010, 2012 and 2014. collected by ielts-nguyenhuyen Talley BO sd itech 2014) Passenger 11,800,000 12,700,000 13,000,000 +10.2% vehicles Commercial 2,300,000 2,600,000 2,700,000 +17.4% vehicles Motorcycles 540,000 680,000 709,000 +30.8% Heavy trucks 384,000 390,000 416,000 +8.3%"],
    ["Total population (millions) Male population (%) Female population (%) Birth rate (%) Average annual population growth (%) Population aged over 65 (%) Australia 1980 14.7 49.9 50.1 9.6 2002 19.6 49.9 50.1 1.3 1.3 12.4 1980 13.7 50.3 49.7 3.2 24 3.7 2002 50.6 49.4 2.2 21"], 
    ["The table below gives information about the value of exports of kiwi fruit from New Zealand to five countries between 2010 and 2012 Value (NZS) of exports of kiwi from New Zealand, 2010-2012     2010 2011 2012 Japan $271,100,000 $287,400,000 $325,300,000 China $74,900,000 $84,800,000 $94,000,000 Mexico $6,000,000 $2,400,000 $3,300,000 Russia $968,000 $1,585,000 $2,404,000 Saudi Arabia $290,000 $106,000 $82,000"],
    ["The table below shows the percentage of the population who rode bicycles in one town by age group in 2012. Summarise the important information by selecting and reporting the main features, and make comparisons where relevant. 0-9 52.5 51.2 10-19 43.6 25.1 20-39 18.2 10.8 40-59 13.7 9.3 60+ 19.8 14.6"],
    ["Participation in cultural activities, by age 16-24 % Any performance* 35 Undertaking any crafts 11 Cultural purchases 11 Any visual arts 30 Any writing 17 Computer based 10 25-44 % 22 17 17 16 * Dancing, singing, playing musical instruments and acting 45-74 % 17 22 18 11 All aged 16 and over % 22 19 16 30 16 11 15 17 6 5 7 10 9 5 6"],
    ["The table below describes the number of employees and factories in England and Wales from 1851 to 1901. Male Female Total ‘ employees employees employees Factories 1851 287,100 190,000 477,100 225 1861 131,780 160,000 291,780 227 1871 80,123 60,000 140,123 622 1881 76,132 50,000 126, 132 721 1891 65,000 40,000 105,000 625 1901 31,000 30,000 61,000 600"],
    ["Participation in cultural activities, by age 16-24 % Any performance* 35 Undertaking any crafts 11 Cultural purchases 11 Any visual arts 30 Any writing 17 Computer based 10 25-44 % 22 17 17 16 * Dancing, singing, playing musical instruments and acting 45-74 % 17 22 18 11 All aged 16 and over % 22 19 16 30 16 11 15 17 6 5 7 10 9 5 6"],
    ["Sales of Fairtrade-labelled coffee and bananas (1999 & 2004) Coffee 1999 2004 UK 1.5 20 Switzerland 3 6 Denmark 18 2 Belgium 1 17 Sweden 0.8 1 Bananas UK 15 47 Switzerland 1 5.5 Denmark 0.6 4 Belgium 1.8 1 Sweden 2 0.9"],
    ["country Tobacco Footwear Education ireland 28.91% 6.43% 221% Italy 16.36% 9.00% 3.20% Spain 18.80% 6.51% 1.98% Sweden 15.77% 5.40% 3.22% Turkey 32.14% 6.63% 4.35%"],
    ["Year 1980 Aspects Good Neither good Nor Bad Education 2 23 5 Healthcare 82 10 8 Environment 72 6 Shops 64 24 12 Employment 2 22 10 Transport 52 16 32 Year 2010 Aspects Good Neither good Nor Bad Education 82 11 9 Healthcare 74 14 2 Environment 71 13 16 shops 60 24 16 Employment 72 5 23 Transport 39 23 38" ],
    ["country Tobacco Footwear Education ireland 28.91% 6.43% 221% Italy 16.36% 9.00% 3.20% Spain 18.80% 6.51% 1.98% Sweden 15.77% 5.40% 3.22% Turkey 32.14% 6.63% 4.35%"],
    ["Reasons for attending the arts - UK adults (all numbers below are percentages) 20 years ago 10 years ago today Desire to see specific performer/artist 58 55 56 Accompanying children 9 15 24 Special occasion/celebration 7 20 9 Work/business 6 10 11"],
    ["Independent films released in the UK and Republic of Ireland by genre 2012 Genre Number of % of all releases % of income from releases ticket sales Comedy 26 17.6 45.4 Horror 14 9.5 20.2 Biopic 1 0.7 9.1 Drama 35 23.6 8.3 Crime 7 4.7 4.7 Action 4 2.7 4.1 Documentary 35 23.6 2.9 Thriller 13 8.8 1.3 Romance 5 3.4 0.8 Other 8 5.4 3.2 Total 148 100 100"],
    ["Languages with the most native speakers Language Number of native speakers Number of speakers as an aditional language Total number of speakers Mandarin Chinese 900 million 190 million 1,090 million Hindi 370 million 120 million 490 million Spanish 350 million 70 million 420 million English 339 million 603 million 942 million Arabic 206 million 24 million 230 million Portugese 203 million 10 million 213 million"],
    ["Average Vancouver Vancouver Vancouver Georgia Street Dunsmuir Street Drake Street Number of Transactions 854 750 390 Transaction Value $10.69 $5.61 39.02 Number of Eatin 405 58 200 Transactions Number of Take-Away 449 392 190 Transactions Most popular Item Chicken Burger Chicken Wings Chicken Pasta"],
    ["Sales of Fairtrade-labelled tea and pineapples (2010 & 2015) Tea 2010 2015 (millions of euros) (millions of euros) France 25 21 Austria 4 8 Germany 28 3 Netherlands 2 27 Norway 18 2 Pieapples (millions of euros) (millions of euros) Austria 16 48 France 2 65 Netherlands 16 5 Norway 28 2 Germany 3 1.9"],
    ["Participation in cultural activities, by age 16-24 % Any performance* 35 Undertaking any crafts 11 Cultural purchases 11 Any visual arts 30 Any writing 17 Computer based 10 25-44 % 22 17 17 16 * Dancing, singing, playing musical instruments and acting 45-74 % 17 22 18 11 All aged 16 and over % 22 19 16 30 16 11 15 17 6 5 7 10 9 5 6"],
    ["Company 2006 % Market Share 2007 % Market Share HP. 31.4 34 Dell 16.6 20.2 Acer 11.6 10.7 Toshiba 6.2 7.3 Lenovo 6.6 6.2 Fujitsu-Siemens 4.8 2.3 Others 22.8 19.3 Total 100 100"],
    ["Qualifications for English Language Teachers obtained 2007/8 and 2008/9, UK 2007/8 2008/9 Total TEFL Cambridge UCLES CELTA & other degrees Total TEFL Cambridge UCLES CELTA & other degrees Total 32,930 25,446 7,484 32,945 24,917 8,028 Female 23,842 18,460 5,382 24,324 18,446 5,878 Male 8,165 6,870 1,295 7,511 6,545 966 % Male 24.7% 26.9% 17.3% 22.7% 26.2% 12.1%"],
    ["Company 2006 % Market Share 2007 % Market Share HP. 31.4 34 Dell 16.6 20.2 Acer 11.6 10.7 Toshiba 6.2 7.3 Lenovo 6.6 6.2 Fujitsu-Siemens 4.8 2.3 Others 22.8 19.3 Total 100 100"],
    ["Participation in cultural activities, by age 16-24 % Any performance* 35 Undertaking any crafts 11 Cultural purchases 11 Any visual arts 30 Any writing 17 Computer based 10 25-44 % 22 17 17 16 * Dancing, singing, playing musical instruments and acting 45-74 % 17 22 18 11 All aged 16 and over % 22 19 16 30 16 11 15 17 6 5 7 10 9 5 6"],
    ["Cinema viewing figures for films by country, in millions Action Romance Comedy Horror India 8 7.5 6.5 2.5 Ireland 7.6 3.8 5.5 6.4 New Zealand 7.2 4.5 3.9 4.7 Japan 7.1 4.5 4 2.2 Total 29.9 20.3 19.9 15.8 Totals 24.5 23.3 20.3 17.8"],
    ["People’s reasons for using the car in city Commute to work 55% Shopping 15% Leisure activities 15% Taking children to school 40% Business 45%"],
    ["Number of children ever born, Women aged 40-44 years Four or One Two Three more None child children children children year % % % % % 2006 15.9 13.2 38.3 21.5 11.0 1996 12.8 11.3 38.2 24.6 13.1 1986 97 8.7 35.6 27.0 18.9 1981 8.5 7.6 29.0 27.4 27.6"],
    ["All students, Age up to 22 years Age 23-25 years Age over 26 years Very organised % 21 18 26 35"],
    ["1991 2001 Australian Dollar Per Month Food 155 160 Electricity and Water 75 120 Clothing 30 20 Housing 95 100 Transport 70 45 Other goods and services* 250 270 Total 675 715"],
    ["reasons for shopping at ASDA supermarket no. of men no. of women close to home 25 20 good reputation 17 22 24-hour shopping 4 0 parking facilities 21 20 friendly staff 6 6 competitive prices 14 17"],
    ['London 2012 Olympic Games Medal Table Rank by Gold Country United States China Great Britain Russia South Korea Germany France Italy Hungary Australia Gold 46 38 29 24 13 11 11 8 8 7 Silver 29 27 17 26 8 19 11 9 4 16 Bronze 29 23 19 32 7 14 12 11 5 12 Total 104 88 65 82 28 44 34 28 17 35'],
    ["German French Indonesian Malay Listening Reading Writing Speaking Overall 6.8 6.3 6.6 6.9 6.7 6.3 6.1 6.5 6.6 6.5 6.3 6.1 6.1 6.7 6.3 6.2 6.4 6.0 6.6 6.4"],
    ["Beer* Fruite juice* 1 Ireland 155 litres 1 Canada 52.6 litres 2 Germany 119 litres 2 United States 42.8 litres 3 Austria 106 litres 3 Germany 38.6 litres 4 Belgium 98 litres 4 Austria 37.3 litres 5 Denmark 98 litres 5 Sweden 35.5 litres 6 United Kingdom 97 litres 6 Australia 34.4 litres 7 Australia 89 litres 7 Finland 33 litres, 8 United States 85 litres 8 United Kingdom 29.3 litres 9 Netherlands 80 litres 9 Netherlands 28.1 litres 10 Finland 79 litres 10 New Zealand 24.8 litres"],
    ["From 30 - 50 years old Tv Sport Reading Hobbies Music Beach Sleep Canada 60 22 15 40 3 0 2 France / / 30 20 4 / / England / / 30 21 4 / 20 Australia 65 30 15 45 5 30 4 Korea 22 21 60 45 2 2 4 China 15 25 60 50 0 5 5 USA 60 23 15 42 23 30 2 Japan / / 62 / / / /"],
    ["Country Population Irrigated land Water consumption per person Brazil 176 million 26,500 km2 359 m3 Democratic Republic of Congo 5.2 million 100 km2 8 m3"],
    ["Weekly rents per property (£/w) Salaries needed (£/year) Area 1 bed 2 bed 3 bed 1 bed 2 bed 3 bed Notting Hill 375 485 738 98,500 127,500 194,000 Regent's Park 325 450 650 85,500 118,000 170,500 Fulham 215 390 600 56,500 102,500 157,500"],
    ["Family type single aged person aged couple single, no children couple, no children sole parent couple with children all households Proportion of people from each household type living in poverty 6% (54,000) 4% (48,000) 19% (359,000) 7% (211,000) 21% (232,000) 12% (933,000) 11% (1,837,000)"],
    ["Underground Railways Systems City Date opened Kilometres of Passengers ay year London 1863 394 775 Paris 1900 199 1191 Tokyo 1927 155 1927 Washington DC 1976 126 144 Kyoto 1981 1 45 Los Angeles 2001 28 50"],
    ["Sales: week of October 7-13 Mon. Tues. Wed. Thurs. Fri. Sat. Sun. Lunch $2,400 $2,450 $2,595 $2,375 $2,500 $1,950 $1,550 Dinner $3,623 $3,850 $3,445 $3,800 $4,350 $2,900 $2,450"],
    ["NUMBER OF VISITORS (million) 1975 2005 SOUTH KOREA 2.9 9.1 JAPAN 3.2 12.0 CHINA 0.3 0.8 USA 0.4 1.1 BRITAIN 0.9 2.9 EUROPE 1.1 4.5 Total 8.8 30.4"],
    ["Consumer durables 1972 1974 Percentage of households with: central heating television video vacuum cleaner refrigerator washing machine dishwasher telephone 39 93 87 73 66 42 43 95 89 81 68 50 1976 48 96 92 88 71 54 1978 52 96 92 91 75 3 60 1979 55 97 93 92 74 3 67 1981 59 97 94 93 78 4 75 1982 60 97 95 93 79 4 76 1983 64 98 18 94 80 5 77"],
    ["Hours of leisure time per year in Someland Teens 20s 30s 40s 50s 60s 70s+ Watching 1,200 700 400 500 600 700 1,100 TV/videos Socialising with 150 150 300 250 250 200 200 4 or less people Socialising with 350 350 50 50 25 25 25 4 or more people Individual 150 100 200 200 50 75 150 exercise Group 450 350 200 150 50 0 0 exercise/sport Cinema 100 75 50 25 25 50 75"],
    ["Occupation Student TruckDriver Fulltime Business mother Executive Doctor"],
    ["Indicators Canada Japan Peru Zaire Annual income per person (in $US) 11100 15760 160 130 Life expectancy at birth 76 78 51 47 Daily calorie supply per person 3326 2846 1927 1749 Adult literacy rate (%) 99 99 68 34"],
    ["Foreign students in English-speaking countries Number of students in 1000s 2004 2012 USA 175000 245000 UK 120000 200000 Australia 120000 200000 Canada 41000 61000"],
    ["Percentage of Canadian workforce in each industry 1850 2020 Manufacturing 36% 2% Services 33% 81% Argriculture/Fishing 22% 1% Construction 5% 8% Energy/Water 3% 1%"],
    ["Average Daily Sales Number of servings Winter Summer Hot coffee 75 30 Iced coffee 2 40 Ice cream 25 35 Soup 50 25 Salad 25 25"],
    ["The age of the population of iceland between 1990 and 2020 Percentage 1990 2000 2010 2020 65 and over 11 12 15 15 55-64 12 11 11 10 25-54 31 34 39 49 15-24 21 19 14 8 0-14 25 24 21 18"],
    ["Female school-leavers going into higher education Percentage 1980 2015 UK 35% 55% USA 38% 58% Australia 58% 58% South Korea 15% 30% France 50% 54%"],
    ["Number of male and female participants in the Olympics Year 1924 1952 1984 2012 Men 3000 4500 5100 5900 Women 100 500 1500 4800"],
    ["The percentage of part-time workers in each country of the United Kingdom Year 1980 2010 England 25% 31% Northern Ireland| 15% 14% Scotland 11% 19% Wales 33% 35%"],
    ["Inductries as a percentage of Brazil's economy Percentage 2009 2019 2029 Finance 31% 37% 42% Food processing 11% 21% 11% Manufacturing 22% 16% 5% Tourism 26% 21% 21%"], 
    ["Population and weather shares by region Percentage Population Share| Wealth share North America 6% 34% Latin Ameraca and Caribbean 8% 4% Europe 15% 29% Africa 10% 1% China 23% 16% India 5% 23%"],  
    ["60 Percentage of Canadian donors by age group: 2000 and 2015 Percentage 2000 [| 2015 18 to 24 20% 11% 25 to 34 29% 25% 35 to 49 39% 34% 50 to 64 34% 38%"],
    ["Employment figures in different tourism-related industries between 2009 and 2019 Total employment (in thousands) 2009 2019 Sports 284.7 345.7 Culture 82.8 774 Travel 60.9 114.2 Food 283.4 357.7"], 
    ["Main reasons for working from home, 2019 Percentage Males Females To save money 45% 42% More productive 24% 11%"],
    ["Hours worked from home, 2019 Percentage Males Females Under 10 3% 74% 10 to 30, 16% 22%" ],
    ["Changes in percentage of population in urban areas Percentage 1950 2007 2050 Africa 15% 37% 62% Asia 17% 41% 66% Europe 51% 72% 84% Latin America 42% 76% 89% North America 64% 79% 90%"],
    ["The places visited by different people living in Canada. Percentage Cinema | Theatre | Library | Zoo New migrants born in other countries 50% 10% 55% 35% New migrants born in English-speaking countries| 65% 25% 40% 50%"],
    ["Percentage of people using multiple social networking sites Percentage 2014 2015 No sites 22 21 One sites 36 28 Two sites 23 24 Three sites 12 16 Four sites 5 8"], 
    ["Someland's main exports USD billion 2005 2015 2025 Meat 4 3.5 3 Dairy 5 8 76"],
    ["Sports played in New Zealand in 2002 Percentage Girls Boys Cricket 1% 10% Alhletics 5% 4% Martial arts 3% 10% Basketball 6% 7% Tennis 7% 8% Netball 25% 1% Soccer 5% 24% Swimming 23% 13%"],
    ["Science qualification in two countries Percentage Malaysia [Singapore Master's degree 3% 3% Bachelor's degree 10% 20% School leaving exams | 30% 35%"],
    ["Students passing high school competency exams, by subject and gender, 2010-2011 Percentage Girls Boys Physics 36.7% | 34.6% Chemistry 30.2% | 14.1% History 25.6% | 22.9% Geography 20.1% | 30.4% Foreign languages 47.4% | 46.8% Mathematics 49.2% | 48.4%"], 
    ["Airport visitors, 1995-2000 Millions of passengers per year 1995 1996 1997 1998 1999 2000 John F. Kennedy 26 35 47 35 32 44 LaGuardia 35 40 42 46 50 68"],
    ["Female unemployment rates in the four countries of the UK Percentage 2013 2014 England 6.8% 6.5% Wales 5.4% 5.0% Scotland 6.1% 6.7% Northern Ireland 5.6% 4.6%"],
    ["Shares of expenditures for selected categories, United States, Canada, United Kingdom, and Japan, 2009 Percentage United States Canada United Kingdom Japan Food 14% 15% 20% 23% Housing 26% 21% 24% 22% Transportation 17% 20% 15% 10% Health care 7% 4% 1% 4% Clothing 4% 7% 6% 4%"],
    ["International graduates, Canadian universities, 2001 and 2006 Percentage 2001 2006 New Brunswick 7% 12% Nova Scotia 7% 7% Quebec 6% 8% Alberta 5% 4% British Columbia 4% 11% Manitoba 3% 6% Newfoundland 3% 6% Ontaria 3% 7%"],
    ["Percentage of population aged 65 and over Percentage 1985 2035 England 15% 23% Wales 16% 26% Scotland 14% 25% Northern Ireland 12% 23% UK 15% 23%"],
    ["Percentage of population in urban areas Percentage 1950 2007 2030 World 29% 49% 60% Africa 15% 37% 51% Asia 17% 41% 54% Europe 51% 72% 78% Latin America 42% 76% 84% North America 64% 79% 87%"],  
    ["The percentage of male and female teachers in six different types of educational setting in the UK in 2010 Percentage Men | Women Nursery 3% 97% Primary 7% 93% Secondary 48% 52% College 50% 50% Private training 52% 48% University 70% 30%"],
    ["Home and International students, 2010-2012 Number of students 2010 2011 2012 Men British 40 24 43 Men International 20 24 39 Women Britsih 32 44 45 Women International 14 17 20"],
    ["Estimsted maximum population of Asian elephants Number of elephants 1997 2004 Vietnam 1200 1100 India 9900 7500 Malaysia 3000 1100 Sri Lanka 3000 2800 Cambodia 1200 1300 Laos 1200 1200 Myanmar. 5200 4800 Thailand 3800 1200 China 1000 500"],
    ["GDP growth Annual percentage 2007 2008 2009 2010 Tunisia 6.3% 4.5% 3.1% 3.0% Japan 2.0% 4.1% 5.2% 6.5% Ecuador 3.1% 4.9% 1.5% 2.3%"],
    ["Total population age 65 and over Percentage 65-74 75-84 854 1900 2.9% 1.0% 0.2% 1910 3.0% 1.1% 0.2% 1920 3.3% 1.2% 0.2% 1930 3.8% 1.3% 0.3% 1940 4.8% 1.7% 0.3% 1950 5.6% 2.2% 0.4% 1960 6.1% 2.6% 0.5% 1970 6.1% 3.0% 0.7% 1980 6.9% 3.4% 1.0% 1990 7.3% 4.0% 1.2% 2000 6.5% 4.4% 1.5%"],
    ["Integration challenges (%) age group Average percentage number of reported challenge pe person 18-34 35-54 Over 55 Learning the local language 46% 36% 23% Finding somewhere to live 40% 39% 22% Making friends 29% 35% 54%"],
    ["Girls enrolled in different levels of school education The number of girls per 100 boys 1990 1998 Primary Developing Countries 83 87 Primary Developed Countries 95 96 Secondary Developing Countries 72 82 Secondary Developed Countries 98 99 Tertiary Developing Countries 66 75 Tertiary Developing Countries 105 112"],
    ["Proportions of the world’s oil resources held in different areas, together with the proportions consumed annually in the same areas Percentage Total world oil resources Total world annual oil consumption Middle East 56.52% 6.15% Canada 14.84% 2.48% [America 8.12% 6.70% Eastern Europe 6.58% 6.56% Africa 6.38% 3.35% Asia 2.89% 26.21% USA 1.85% 25.48% Western Europe 1.47% 19.24% Australia 0.30% 1.31% Other 1.05% 2.52%"],
    ["Division of household tasks : by gender, Great Britain Minutes per person per day Males Females Cooking baking washing 30 74 Cleaning house tidying 13 58 Gardening pet care 49.5 24 Care of own children 20 45 Maintainance odd jobs 26 9.5 Clothers ironing sewing 2 25"],
    ["Student expenditure (age under 26 in higher education) United Kingdom Percentage of total expenditure 1999 1996 Accommodation 20% 23% Food, bills household goods 18% 20% Essential travel 6% 4% Non-essential travel 3% 4% Children 0% 1% Course expenditure 7% 10% Entertainment 31% 26% Other 16% 12%"],     
    ["The preferred leisure activities of Australian children Percentage Boys age 5-14 Girls age 5-14 Skateboarding or rollerblading 35% 25% Bike riding 70% 60% Watching TV or videos 100% 100% Electoronic or computer games 80% 60% Art and craft 60% 35%"],
    ["Factors affecting work performance Percentage 18-30 45-60 Team spirit 60% 60% Chance for personal development 90% 40% Relaxed working environment 80% 30% Competent boss 50% 50% Job security 40% 20% Respect from colleagues 40% 45% Promotion prospects 80% 50% Job satisfaction 50% 50% Work environment 30% 30% Money 75% 70%"],
    ["Estimated world illiteracy rate, by region and by gender, 2000 Percentage Male Female Developed Countris 1% 2% Latin America/Caribbean 10% 12% East Asia/Oceania 8% 20% Sub-Saharan Africa 31% 48% Arab States 29% 52% South Asia 34% 56%"],
    ["Mobile Phones and Land Lines, per 100 people Number of phones per 100 people Mobile Phones [Land Lines Canada 40 65 us 50 70 Germany 71 60 Denmark 83 90 UK 84 58 Sweden 87 67 Italy 88 45"],
    ["Internet Usage in Taiwan by Age Group, 1998-2000. Percentage 1998 1999 2000 Less than 15. 2% 8% 9% 16-30 years 53% 45% 44% 31-50 years 41% 39% 37% 50 or more 4% 8% 10%"],
    ["Higher Colleges of Technology graduates by College Number of students Number of students Al Ain Men's 131 Al Ain Women's 218 Abu Dhabi Men's 273 Abu Dhabi Women's 293 Dubai Men's 426 Dubai Women's 547 Ras Al Khaimah Men's 102 Ras Al Khaimah Women's 178 Sharjah Men's 165 Sharjah Women's 186"],
    ["Heart Attacks per year, USA (Thousands) Men Women 29-44 123 3 45-64 424 136 65+ 440 374"],
    ["The amount spent on six consumer goods in four European countries. thousand pounds sterling Britain France Italy | Germany Photographic film a7 166 156 146 Toys 168 158 158 148 CDs 162 158 152 146 Perfumes 160 146 153 148 Tennis recquets 153 144 152 150 Personal stereos 153 148 150 148"],
    ["Fertility rate (Birth per women) Gulf Countris 1990-2000 Number of birth 1990 2000 Bahrain 3.5 25 Kuwait 45 2.8 Qatar 42 Bi: Oman 7 5.5 S.Arabia 6.8 5.5 UAE 4 3"],
    ["Expenditure on Fast Foods by Income Grouops, UK 1990 pence per person per week Hamburger [Fish and Chips| Pizza High Income 42 16 19 |Average Income 32 25 12 Low Income 14 16 7"],
    ["Mode of transport and years Percentage of commuters 1960 1980 2000 Train 20% 26% 23% Car 5% 23% 40% Tube 27% 22% 25% Bus 35% 27% 15%"],     
    ["Computers and Internet in the Arab World per 1000 inhabitants Computers Internet Egypt 20 20 Jordan 70 80 Kuwait 130 100 Lebanon 60 75 Morocco 20 20 Oman 80 90 S.Arabia 60 20 Syria 20 5 UAE 150 350"],
    ["The average hours of unpaid work per week done by people in different categories hours per week Married women | Maried men Without children 30 18 With 1-2 children 52 18 With 3+ children 56 16"],
    ["Global sales of games software, CDs and DVD or video billion 2000 2001 2002 2003 Games software 13 15 7 20 DVD/video 18 21 26 30 CDs 35 34 33 32.5"],
    ["Energy capacity in 2015(6.688 GW) and 2040(11.678 GW) Percentage of energy capacity 2015 2040 Fossil Fuels 64% 44% Wind 5% 12% Nuclear 6% 5% Solar 2% 18%"],
    ["Employment in the UK Number of employment 1988 2012 Men (full-time) 12,539 13,794 Men (part-time) 1,550) 2,131 Women (full-time)  4,482) 7,696 Women (part-time) 5,268 5,979"],
    ["World, Projected production by sector Percentage of production by sector 2040 2060 Services 40% 41% Energy 7% 5% Manufacturing 31% 33% Agriculture 4% 1% Materials 3% 4% Info-Com Tech 15% 16%"],
    ["Ways of accessing the news in Canada and Australia. Percentage of the ways of accessing the news Canada  Australia TV 40% 37% Radio 7% 2% Print 14% 7% Online 36% 52% Not specified 3% 2%"],
    ["Employment sectors of graduate from Brighton University, 2019 Percentage of employment sectors 2019 Service Industrial 33.0% Manufacturing Industry| 16.3% Politics & Government | 12.1% Education 14.7% Civil service 5.6% Charity 0.3% Science & Technology 7.3% Sport 0.1% Transportation 7.8% Other 2.8%"],
    ["The devices people in the 18 to 25 age group use to watch television in Canada in 2009 and 2019. Percentage of people use difference device to watch television 2009 2019 Mobile phone 15% 26% Desktop computer 18% 12% Laptop 20% 12% Flat-screen TV 8% 27% Tablet 5% 19% Conventional TV 34% 4%"],
    ["World passenger car production, 2015 Percentage of car production 2015 Greater China 26% Japan/ Korea 16% Europe 25% South Asia 8% North America 19% South America 4% Middle East/Africa 2%"],
    ["515,600 Canadians aged between 25 and 35 and 213,400 Canadians aged between 45 and 55 hold a degree. Their main fields of study were. Percentage of main fields of study 25-35 years|45-55 years Administration 19% 15% Social Science, Arts 20% 21% Education 15% 13% Science, Maths, Computers 18% 10% Engineering 9% 19% Humanities 4% 5% Nursing 7% 11% MBA 6% 3% Other 2% 3%"],
    ["Government Expenditure in 2010 and 2015 Percentage of expenditure 2010 2015 Education 24% 21% Healthcare 21% 20% Pensions 19% 19% Defence 17% 14% Welfare 8% 13% Interest on borrowing 5% 10% Transport 3% 1% Culture and leisure 2% 1% Other 1% 1%"],
    ["The sources of electricity produced in 4 countries between 2003 and 2008. Percentage of the sources of electricity produced India _| Sweden | Morocco | Vietnam Fossile fuel 82% 4% 5% 56% Hydro power 14% 52% 95% 44%"],
    ["Number of Journal articles read by each group of student Percentage of journal articles 1to5 6to11 12+ All students 67% 21% 12% Junior lecturers 1% 75% 24% PhD students 5% 15% 80%"],
    ["Employment patterns in Great Britain in 1992. Percentage of employment patterns Female Male Craft or similar 3% 24% General labourers 1% 2% Manageerial and professional 29% 36% Clerical or related 31% 6% Other manual 27% 26% Other non-manual 9% 6%"],
    ["Employment patterns in Great Britain in 1992. Percentage of employment patterns Female Male Craft or similar 3% 24% Gerneral labourers 1% 2% Managerial and professional] 29% 36% Clerical or related 31% 6% Other manual 27% 26% Other non-manual 9% 6%"],
    ["Online sales for retail sectors in Canada Percentage of online sales 2005 2010 Electronics & Appliance 35% 30% Home Furnishings 25% 15% Food & Beverage 22% 32% Video games 18% 23%"],
    ["Expenditure by local authorities in Someland Percentage of expenditure 1980 1990 2000 Higher Education 35% 45% 40% Transportation 16% 6% 22% Environment services 4% 5% 9% K-12 Education 25% 20% 18% Health and human resources 14% 20% 10% Other 6% 4% 1%"],
    ["World population by region, 1900 and 2000 Percentage of world population 1900 2000 Asia pacific 60% 54% Europe 25% 14% North America 5% 5% Africa 4% 10% Latin America & Caribbean 3% 8% Middle East and North Africa 0% 6% Other 3% 3%"],
    ["Average Household Expenditures by Major Category Percentage of average household expenditures 1950 2010 Housing 72.1% 22.0% Food 11.2% 34.0% Health care 2.4% 4.5% Education 6.6% 6.3% Transportation 3.3% 14.0% Other 4.4% 19.2%"],
    ["Spending habits of people in UK between 1971 and 2001. Percentage of spending habits 1971 2001 Cars 43% 22% Food 14% 44% Books 1% 6% Computers 12% 2% Furniture 8% 9% Restaurant 14% 7% Petrol 8% 10%"],
    ["Comparison of Energy Production Percentage of energy production 1995 2005 Gas 29.63% 30.31% Petro 29.27% 19.55% Coal 29.80% 30.94% Nuclear 6.40% 10.10% Other 4.90% 9.10%"],
    ["Household income distribution Percentage of different catagories Food 25% Cloths 13% Education 22% Power 5% Transport 12% Other 8% Saving 15%"],
    ["The main sources of energy in the USA in the 1980s and the 1990s. Percentage of sources of energy 1980 1990 Oil 42% 33% Natural Gas 26% 25% Coal 22% 27% Hydroelectric power 5% 5% Nuclear power 5% 10%"],
    ["US spending patterns between 1966 and 1996 Percentage of spending patterns 1966 1996 Food 44% 14% Cars 23% 45% Petrol 9% 8% Restaurants 7% 14% Furniture 10% 8% Computers 1% 10% Books 6% 1%"],
    ["Highest level of education of women in someland - 1945 and 1995 Percentage of highest level of education of women 1945 1995 No Schooling 35% 0% Third Grade 35% 0% Year 6 15% 0% Year 9 10% 10% Year 12 4% 20% First degree 1% 50% Post graduate 0% 20%"],
    ["The proportion of Australian secondary school graduates who were unemployed, employed or further education in 1980, 1990, and 2000. Percentage of the proportion of Australian secondary school graduates 1980 1990 2000 Unemployed 10% 12% 8% Employed 40% 50% 55% Further education 50% 38% 37%"],
    ["How much money people from Japan and Malaysia spend on different sorts of goods and services Percentage of money Japan Malaysia Housing 21% 34% Transport 20% 10% Food 24% 27% Health care 6% 3% Other goods and service 29% 26%"],
    ["Average Household Expenditure in 2019 and 2020 Percentage of average household expenditure 2019 2020 Food 55% 41% Housing 20% 19% Healthcare 5% 20% Education 13% 2% Other products and services 7% 18%"],
    ["The results of a questionnaire that asked visitors to the Parkway Hotel how they rated the hotel’s customer service Percentage of results of a questionnaire 2005 2010 Excellent 5% 28% Satisfactory 45% 17% Good 14% 39% Poor 21% 12% Very poor 15% 4%"],
    ["The proportions of British students at one university in England Percentage of British students 2000 2010 No other language 20% 10% French 15% 10% German 10% 10% Spanish 30% 35% Another language 15% 20% Two other language 10% 15%"],
    ["Time spent on smartphones and tablets, by category Percentage of time spent Smartphones| Tablet Games 35% 57% Social Network 29% 15% Utilities 20% 3% Music and videos 8% 13% News 3% 4% Other 5% 8%"],
    ["Various career and academic choices taken by high school graduates in Lakeside Country in 2010 and 2020 Percentage of career and academic choices 2010 2020 No idea 5% 15% Working 24% 10% Starting a business 5% 24% Working (freelance) 11% 13% Going to college 55% 38%"],
    ["The main methods of transport of people travelling to one university in 2004 and 2009. Percentage of the methods of transport 2004 2009 Car 51% 28% Bicycle 9% 16% Bus 33% 46% Train 3% 4% Walking 4% 6%"],
    ["The proportions of energy produced in a country from 1983 to 2003. Percentage of the proportions of energy produced 1983 2003 Hydropower 4% 4% Oil 42% 38% Nuclear power 20% 24% Coal 23% 13% Nutural gas 11% 21%"],
    ["The proportion of people’s total spending in a particular European country was spent on different commodities and services in 1958 and in 2008. Percentage of people’s total spending 1958 2008 Housing 22% 32% Food 32% 12% Clothing 18% 16% Travel/transport 8% 17% Entertainment 13% 6% Luxuary goods 7% 17%"],
    ["The results of a survey into the most popular leisure activities in the USA in 1999 and 2009. Percentage of the most popular leisure activities 1999 2009 Soccer 17% 18% Camping 8% 9% Swimming 9% 18% Aerobics 13% 4% Walking 29% 31% Jogging 7% 3% bicycling 12% 7% Yoga 5% 10%"],
    ["The expenditure involving in publishing a magazine. Percentage of expenditure Editorial content development 30% Promotion costs 18% Binding costs 12% Paper costs 10% Transportation 6% Printing costs 24%"],
    ["What makes people happy the most? Percentage of the cause of happy Under 30| Over 30 Achievement at work 31% 32% Having a good appearance 18% 20% Travlling 15% 14% Doing hobbies 22% 24% Other factor 14% 10%"],
    ["Online sales for retail sectors in Canada. Percentage of online sales 2005 2010 Electronics&Appliance 35% 30% Home Furnishing 25% 15% Food&Beverage 22% 32% Video games 18% 23%"],
    ["Ways of delivering the news in Canada and Australia Percentage of delivering the news Canada | Australia TV 40% 37% Perint 14% 7% Radio 7% 2% Online 36% 52% Not specific 3% 2%"],
    ["The gadgets people use to watch television in Canada. Percentage of gadgets to watch television 2009 2019 Mobile phone 15% 26% Tablet 5% 19% Flat-screen TV 8% 27% Laptop 20% 12% Desktio computer 18% 12% Conventional TV 34% 4%"],
    ["Government expenditure in 2010 and 2015 Percentage of government expenditure 2010 2015 Education 24% 21% Healthcare 21% 20% Pensions 19% 19% Defence 17% 14% Welfare 8% 13% Interest on borrowing 5% 10% Transport 3% 1% Culture and leisure 2% 1% Other 1% 1%"],
    ["The visitors' responses to a survey about customer service at the Parkway Hotel in 2005 and in 2010. Percentage of customer service 2005 2010 Excellent 5% 28% Satisfactory] 45% 17% Good 14% 39% Poor 21% 12% Very poor 15% 4%"],
    ["The percentage of world forest and also the percentage of timber in five different regions. Percentage of each region world forest|_ timber South America 16% 23% Africa 27% 9% Asia 14% 18% Europe 18% 20% North America 25% 30%"],
    ["The living arrangements of two different age groups in a certain country in 2015. Percentage of living arrangements aged 25-34|aged 35-44 Living alone 13% 19% Living with parents 37% 19% Living with partner/spouse 13% 48% Living with flatmates 37% 14%"],
    ["The proportion of staff employed by age in the public sector in the UK in 2015 and 2020. Percentage of staff employed 2015 2020 Under 21 14% 12% 21-30 23% 33% 31-40 21% 11% 41-50 19% 28% Over 50 23% 16%"],
    ["Energy Usage and Greenhouse Gas Emissions in Australia Percentage of Energy Usage and Gas Emissions Energy Usage|Gas Emissions Water heating 30% 32% Heating 42% 15% Cooling 2% 3% Lighting 4% 8% Refridgeration 7% 14% Other appliance 15% 28%"],
    ["Average percentage of sodium, saturated fats and added sugers in typical meals consumed in th USA Percentage of typical meals consumed in th USA Sodium |Saturated fat| Added suger Breakfast] 14% 16% 16% Lunch 29% 26% 19% Dinner 43% 37% 23% Snacks 14% 21% 42%"],
    ["The proportions of graduates from Brighton University in 2019 entering different employment sectors. Percentage of graduates from Brighton University 2019 Service Industries 33.0% Manufacturing 16.3% Politics & Government 12.1% Education 14.7% Civil service 5.6% Charity 0.3% Science & technology 7.3% Sport 0.1% Transportation and warehousing 7.8% Other 2.8%"],
    ["UK students' responses to the question of to what extent would they describe themselves as financially organised. Percentage of financially organised UK students Not at all organised 4% Not very organised 20% Fairly organised 55% Very organised 21%"],
    ["The percentages of industry sectors' contribution to the economy of Turkey in 2000 and 2016. Percentage of industry sectors’ contribution 2000 2016 Agriculture 24% 14% Government 12% 9% Leisure and hospitality 17% 14% Finance, business and other 5% 8% Healthcare and education 17% 17% Trade, utilities and transportation| 14% 16% Manufacturing 8% 12% Construction 3% 10%"],
    ["The main reasons why students chose to study at a particular UK University in 1987 and 2007. Percentage of reasons 1987 2007 Quality of resource 21% 17% Suitable degree couruses 35% 37% Close to parental 10% 22% Quality of teaching 15% 8% Good sports and social activities 19% 6%"],
    ["frequency of eating at fast food restaurants among people in the USA (2003-2013) % of people 2003 2006 2013 Every day 4% 3% 3% Several times a week 17% 20% 16% Once a week 32% 33% 26% Once or twice a month 30% 25% 33% A few times a year 12% 15% 15% Never 5% 3% 3%"],
    ["Percentage of women giving birth to their first chid Under 19 19-24 25-30 30-34 34-39 Over 40 1966 32% 62% 35% 12% 8% 4% 1986 20% 35% 50% 18% 16% 4% 2006 12% 28% 38% 45% 30% Over 40 4% 4% 6%"],
    ["Brazil Colombia Indonesia Vietnam Coffee production Bar chart 1990 0.7 0.6 0.2 0.1 Millon tones 1995 1 0.5 0.2 0.1 2000 1.5 0.4 0.3 2010 1.9 0.3 0.4 0.3"],
    ["Hours spent on Housework in the UK 1985 1995 2005 Unemployed Women 7 6.5 7 Women with part-time employment 5.5 5 6 Women with full-time employment 2 1.5 2.8 Men with full-time employment"],
    ["Instagram Influencers (in millions) 2018 2021 Cristiano ronaldo 110 320 The rock 80 255 Ariana grande 110 250 Kylie jenner 98 250 Selena gomez 120 240"],
    ["Transport preferences among young people The UK United States France Italy car 60 60 40 75 Motorcycle Bicycle 30 30 10 8 10 10 45 12"],
    ["Percentage of business with a social media presence 2012 2013 2014 2015 2016 Small businesses 28% 30% 35% 30% 48% Medium businesses Large businesses 30% 48% 49% 30% 52% 78% 78% 76% 50% 79%"],
    ["Number of people in the UK eating more than 5 portions of vegetables and fish per day Percentage 2011 2012 2013 2014 2015 2016 2017 Women 20 20 20 27 32 37 30 Children 10 14 13 14 15 14 15 Men 16 17 18 22 24 23 26"],
    ["Distances Travelled per person per week (in kilometers) Bicycle Motorbike Air Bus Rail Total 1990 5 6 1 40 35 100 2000 3 4 5 42 38 110"],
    ["total manufacturing production in percentages Asia, Europe, and the rest of the world. 1840 1880 1920 1960 2000 Asia 50% 30% 20% 25% 45% Europe 40% 45% 45% 38% 28% Rest of the World 10% 25% 25% 38% 33%"],
    ["% of population 65 andover 1980 2000 2030 Cannada 10% 15% 22% Germany 15% 20% 25% UK 13% 15% 18%"],
    ["% of Housholds 1971 1981 1991 2001 No car 46% 40% 32% 30% 1car 35% 42% 36% 46% More than 2 cars 25% 32% 26% 32%"],
    ["Number of people in the UK eating more than main features, and make comparisons where relevant. percentage 2011 2012 2013 2014 2015 2016 2017 Women 20% 20% 20% 28% 32% 37% 30% children 10% 14% 14% 14% 15% 14% 15% Men 18% 18% 18% 22% 23% 23% 27%"],
    ["The bar chart shows the number of visitors to four London museums. June July August September December History museum 410,000 400,000 600,000 380,000 280,000 British museum 420,000 420,000 710,000 580,000 480,000 Nation museum 200,000 180,000 380,000 200,000 180,000 Science museum 430,000 400,000 400,000 500,000 300,000"],
    ["British emigration to selected destinations-2004-2007 Thousands Australia Spain New Zealand |USA France 2004 42 32 23 23 22 2005 38 33 22 19 33 2006 52 31 21 24 26 2007 44 25 20 20 18"],
    ["coffee and tea buying and drinking habitis in five cities in Australia Percentage of city residents Sydney Melbourne Brisbane Adelaide Hobart Bought fresh coffee in last 4 weeks 38% 43% 33% 34% 33% Bought instant coffee in last 4 weeks 46% 48% 46% 50% 52% Went to cafe for coffee or tea in last 4 weeks 62% 63% 68% 48% 63%"],
    ["Higher Education Bar Percentage 2000 2005 2010 Country A 35% 46% 57% Country B 40% 38% 40% Country C 50% 52% 52% Country D 60% 63% 80%"],
    ["Organisations Bar Chart Billion euros Machinery Building Staff training Research Organization A 1 1.5 1.2 1.1 Organization B 0.9 1.4 1.9 1.6 Organization C 1.3 1 0.8 1.3 Organization D 0.9 1 0.7 0.6 Organization E 0.8 1 0.9 0.7"],
    ["Exports in Various Categories Billions Petroleum products Engineered goods Gems and jewellery Agricultural products Textiles 2015 61 55 42 31 25 2016 62 62 41 32 32"],
    ["yearly income year Amandline Bakesry Mari Bakeshop Bolo Cakery 2001 20,000.00 80,000.00 58,000.00 2002 25,000.00 95,000.00 58,000.00 2003 21,000.00 90,000.00 58,000.00 2004 39,000.00 80,000.00 58,000.00 2005 39,000.00 92,000.00 58,000.00 2006 38,000.00 61,000.00 60,000.00 2007 41,500.00 59,000.00 66,000.00 2008 62,000.00 60,000.00 97,000.00 2009 62,500.00 41,000.00 105,000.00 2010 64,000.00 40,000.00 109,000.00"],
    ["unemployment rate in Asia 2012 2013 2014 Phippines 5.5 5.3 5.8 China 41 42 43 Japan 3.4 3.3 4.2 Singapore 2 1.8 1.4 India 47 5.2 46 Thailand 0 1.3 1.2"],
    ["Secondary School Attendance Specialist Schools Grammar Schools Voluntary-controlled Schools Community Schools 2000 2005 2009 12% 11% 10% 24% 19% 12% 52% 38% 20% 12% 32% 58%"],
    ["Gym Membership year men women 1980 2000 1000 1985 4000 1600 1990 3000 1900 1995 1800 2800 2000 2400 2200 2005 5000 2700 2010 800 1800"],
    ["year 1980 1985 1990 1995 2000 2005 2010 2015 Louvre 7.9 8.6 8.7 8.2 8.4 8.5 9 Number of visitors (millions) Vatican Museums 4.6 4.7 5.6 6.2 7 8 8.5 8.8 London Science Musemum Shenzhen Museum 4 4 3.2 3 2.8 2.7 2.2 2.1 3.8 3.5 3.2 3 2.8 2.6 2.4 2.2"],
    ["year People moving from rural to urban areas Number of people (millions) 2000 2005 2010 2015 2020 2025 Iran 10 35 48 73 95 Indonesia 10 13 20 22 30 29 Russia 15 30 40 70 78 86"],
    ["year International student enrolment Number of students 2005 2006 2007 2008 2009 Iran 38 60 70 85 158 Greece 180 158 70 65 42 Turkey 20 55 85 110"],
    ["Free time activities of the elderly in the United states form 1980 to present 1980s 1990s 2000s 2010s Reading Hiking Theatre Watching TV Surfing the Internet 30% 20% 50% 60% 5% 40% 40% 30% 65% 10% 20% 60% 40% 70% 15% 60% 80% 45% 65% 50%"],
    ["Population in India and China, 2000 - 2050 (Billinons) 2000 2010 2020 2030 2040 2050 India 1 1.18 1.31 1.45 1.52 1.6 China 1.25 1.35 1.41 1.45 1.41 1.38"],
    ["Population level in London, micrograms per cubic metre year 1600 1650 1700 1750 1800 1850 1900 1950 2000 Smoke 80 100 180 180 250 350 440 300 0 Sulphur dioxide 85 450 650 700 750 800 720 320 0"],
    ["Burnaby Public Library Number of books read 2011 2012 2013 Men 3000 4000 8500 Women 5000 8000 8500 2014 14000 6000"],
    ["Percentage of tourists to England who visited Brighton attractions Art Gallery Pavilion Pier Festival 1980 22% 23% 8% 30% 1985 38% 25% 15% 25% 1990 20% 35% 7% 23% 1995 21% 43% 12% 23% 2000 20% 35% 11% 25% 2005 15% 32% 15% 27% 2010 8% 30% 22% 27%"],
    ["Outokumpu share price, Euro Decline overall in the share price of Outokumpu 2006 2007 2008 2009 2010 17 25 30 14 13.5"],
    ["year Money spent on Books US dollars (millions) 1995 1997 1999 2001 2003 2005 Germany 80 85 90 87 85 95 France 55 58 60 69 70 75 Italy 50 57 50 55 56 62 Austria 30 36 40 40 60 62"],
    ["Number of visitors per month in summer, 2013 (in thousands) June July August September British Museum 600 750 500 650 Science Musem 400 370 300 430 Natural History Museum 550 380 390 440"],
    ["1960 1965 1970 1975 1980 1985 1990 1995 2000 2005 2011 National Recycling Rate, 1960 to 2011 Total Waste Recovery (million tons) Percent Recycling 5.6 65 80 93 145 16.7 33.2 55.8 69.5 79.8 86.9 6.4% 6.2% 6.6% 7.3% 9.6% 10.1% 16.0% 25.7% 28.5% 31.4% 34.7%"],
    ["year 1995 2000 2005 2010 Barzil Interantional Tourist Arrivals (millions) 9.8 10 11 12 Egypt 9.5 11 12 15 Malaysia 20 23 30 42 France USA 30 60 65 80 70 82 85 80"],
    ["Population of turtles in India: 1980-2012 Index (19880 = 100) leather turules green turtles all species olive ridley turtles 1980 100 100 100 100 1982 102 108 102 102 1984 98 108 101 102 1986 110 105 103 103 1988 100 105 100 105 1990 95 105 99 107 1992 80 100 95 109 1994 80 95 98 111 1996 65 92 94 113 1998 60 100 99 115 2000 70 85 100 117 2002 55 80 103 119 2004 58 82 102 121 2006 55 81 104 123 2008 56 80 103 125 2010 57 78 101 127 2012 56 80 102 129"],
    ["chanel one news viewing figures millions of viewers per day 1pm 6pm 9.30pm 11pm Jan 13 48 3.2 Feb 1.25 45 3.4 Mar 13 3.8 3.5 Apr 1.26 45 3.6 May 13 3.8 3.8 Jun 1.25 42 2.8 Jul 13 3.6 2.2 Aug 13 3.4 1.2 Sep 13 3.2 2 Oct 1.23 4 2.4 Nov 13 3.8 2.6 Dec 1.35 3.2 3 0 0 0 0 0.2 1.8 3.2 4.1 3.2 1.8 1.6 1.4"],
    ["International student enrolment in British universities 2009-2014 Year 2019 2014 Manchester 6 8 Liver pool 8 8 Bimimgham 8 12 Brighton 12 13 Glasgow 4 6 Edingbourog 3 6 Bristol 5 6 Leeds 9 10 Huddersfield 1 3 Cardif 2 1"],
    ["Expected city visits by country of origin for 2018 (Thousands/year) Mexico Cannada USA London 20 45 80 Paris 23 42 100 Madrid 65 25 65 Istanbul 20 63 60"],
    ["Calorie Source for UK males at different life periods. Percentage of total intake 0-24 25-49 Over 50 Dairy 42% 23% 16% Meat 20% 50% 20% Pulses 18% 10% 65% Vegetables 20% 15% 10%"],
    ["average life expectancy per country age Monaco United States Philippines Laos Rwanda South Africa 85 72 70 57 55 48"],
    ["Passenger served per airport terminal Total passengers(2014) Atlanta Beijing Dubai Chicago London Losangele Hong kong 7,600,000 7,500,000 6,000,000 5,800,000 5,800,000 5,800,000 5,800,000"],
    ["Coastline Coverage by country Coastline Coverage (km) Canada Indonesia Russia Philippines Japan 100000 70000 58000 32000 28000"],
    ["Waste Hauling Truck Trip No. of trips Hauler A Hauler B Jul-14 100 85 Aug-14 115 145 Sep-14 82 100"],
    ["Book Sales By Genre Across Time Total No. of Sales 2013 2014 2015 Fantasy 250 720 1300 Romance 420 820 1500 Mystery 410 780 1000 Thriller 415 880 1250 Science fiction 418 900 1300"],
    ["hazardous waste inventory 2014 Inventory in kags Medical waste Expired medicine Wet Batteries Busted Flourescent Hydraulic 1000 1200 2800 3200 4600"],
    ["Active Military Manpower Per country Manpower China Us India Russia North Korea 2,3000,000 1,400,000 1,300,000 700,000 650,000"],
    ["Water service Reading Building A Building B Building C Building D Building E Building F Building G May 24 reading 25000 65000 70000 30000 50000 50000 60000 June 24 reading 30000 65500 72000 40000 59000 55000 65000"],
    ["Power Consumption (Per Localtion) for July 2015 KWH consumption Power cente Passenger T Admin Bldg Triangle IPB Hirring Hall 92000 63000 82000 62000 75000 98000"]
]

In [ ]:
len(Ref_key)

201

In [ ]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 3.9 MB/s 
     |████████████████████████████████| 132 kB 37.2 MB/s 
     |████████████████████████████████| 452 kB 28.0 MB/s 
     |████████████████████████████████| 213 kB 62.7 MB/s 
     |████████████████████████████████| 182 kB 61.1 MB/s 
     |████████████████████████████████| 127 kB 63.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=List_key, references=Ref_key)
print(results)

{'bleu': 0.8316937918975915, 'precisions': [0.959860788863109, 0.9289786223277909, 0.9076642335766424, 0.8901496259351621], 'brevity_penalty': 0.9027404608804743, 'length_ratio': 0.9071774363291939, 'translation_length': 8620, 'reference_length': 9502}


In [ ]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=['cat', 'dog', 'horse'], references=[['cat'], ['dog'], ['horse']])
print(results)

{'bleu': 0.0, 'precisions': [1.0, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 3, 'reference_length': 3}


In [ ]:
predictions = ["i love cat so much","you hat dog"]
references = [
        ["i love cat so miss"],
        ["you hate dog"]
]
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
print(results)

{'bleu': 0.5533409598501607, 'precisions': [0.75, 0.5, 0.5, 0.5], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 8, 'reference_length': 8}


In [ ]:
!pip install fastwer --upgrade
!pip install pybind11

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached fastwer-0.1.3.tar.gz (4.6 kB)
  Created wheel for fastwer: filename=fastwer-0.1.3-cp38-cp38-linux_x86_64.whl size=659197 sha256=fcf40e28cd9c776f80010466c24720e6eb89ab89aac4618a9f7806b6368ce2f1
  Stored in directory: /root/.cache/pip/wheels/87/7c/cc/6ba6edc83d9f67d96c02b4b7d31eb98284100d96389563db7c
Successfully built fastwer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import fastwer

In [ ]:
import numpy as np

In [ ]:
score_char = []
for i in range(len(List_key)) :
    score_char.append(fastwer.score_sent(str(List_key[i]), str(Ref_key[i][0]), char_level=True))

In [ ]:
np.average(score_char)

11.400178109452737

In [ ]:
score_word = []
for i in range(len(List_key)) :
    score_word.append(fastwer.score_sent(str(List_key[i]), str(Ref_key[i][0]), char_level=False))

In [ ]:
np.average(score_word)

15.684418905472635

In [ ]:
np.average(score_word)

/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

In [ ]:
Ref_key[0][0]

'Holiday leisure activities Number of hours At the beach Reading books Chinese tourists 3 8 American tourists 4 5 Turkish tourists 5 4 Brazilian tourists 6 4'

In [ ]:
# Define reference text and output text
ref = 'Holiday leisure activities'
output = 'Holiday leisure activities'

# Obtain Sentence-Level Character Error Rate (CER)
fastwer.score_sent(output, ref, char_level=True)

0.0